In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('./data/NetflixOriginals.csv',encoding='"ISO-8859-1"')
df

,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,Enter the Anime,Documentary,"August 5, 2019",58,2.5,English/Japanese
1,Dark Forces,Thriller,"August 21, 2020",81,2.6,Spanish
2,The App,Science fiction/Drama,"December 26, 2019",79,2.6,Italian
3,The Open House,Horror thriller,"January 19, 2018",94,3.2,English
4,Kaali Khuhi,Mystery,"October 30, 2020",90,3.4,Hindi
...,...,...,...,...,...,...
579,Taylor Swift: Reputation Stadium Tour,Concert Film,"December 31, 2018",125,8.4,English
580,Winter on Fire: Ukraine's Fight for Freedom,Documentary,"October 9, 2015",91,8.4,English/Ukranian/Russian
581,Springsteen on Broadway,One-man show,"December 16, 2018",153,8.5,English
582,Emicida: AmarElo - It's All For Yesterday,Documentary,"December 8, 2020",89,8.6,Portuguese


기본 EDA 

1. 기초통계
2. 결측치 확인
3. 컬럼별 종류 개수
4. 장르별 개수 상위
5. 개봉일 범위
6. 연도별 개봉수
7. 월별 개봉수
8. 장르별 런타임
9. 장르별 스코어
10. 언어별 스코어
11. 언어 개수
12. 다중언어 개수
13. 월별 스코어

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

# 장르 115개

In [ ]:
df.value_counts(df['Genre'])[:20]

# 장르별 개수 상위 20개

In [ ]:
df['Premiere'] = pd.to_datetime(df['Premiere'])
df['Premiere']

# 날짜 정렬

In [ ]:
df.sort_values(by='Premiere')

# 개봉일 범위

In [ ]:
df.groupby(df['Premiere'].dt.to_period("Y")).size()

# 연도별 개봉 개수

In [ ]:
df.groupby(df['Premiere'].dt.month).size()

# 월별 개봉 개수

In [ ]:
df.groupby(['Genre'])['Runtime'].mean().sort_values(ascending=False)[:20]

# 장르별 런타임

In [ ]:
# 장르별 스코어

df.groupby(['Genre'])['IMDB Score'].mean().sort_values(ascending=False).iloc[:10]

In [ ]:
# 언어별 스코어

df.groupby(['Language'])['IMDB Score'].mean().sort_values(ascending=False).iloc[:20]
# 38개 (뒤에 .shape 하면 나옴)

In [ ]:
# 언어 개수

df.value_counts(df['Language'])[:20]

In [ ]:
# 다중 언어 개수

MultiLingual_df = df[df['Language'].str.contains('/')]
MultiLingual_df.groupby(['Language'])['IMDB Score'].mean().sort_values(ascending=False)
# 16개

In [68]:
monthly_basis_score = df.groupby(df['Premiere'].dt.month)['IMDB Score'].mean()

# 월별 스코어

기법 사용 , 제언

월별 score 계산 --> 8월 낮음 --> 분석

In [ ]:
# 8월 가장 낮음

score = np.array([6.0027027027027025,
 6.117948717948717,
 6.322916666666667,
 6.04920634920635,
 6.2132075471698105,
 6.582857142857142,
 6.091176470588234,
 6.481132075471699,
 6.522077922077922,
 6.398245614035088,
 6.2745098039215685])

print(score.mean(),score.std(ddof=1))

one sided ztest 선정

이유
1. 모분산 알고 있음
2. ch_06번 젤리 문제, ch_07번의 수학 문제와 구성이 유사함 (평균 값 비교)
3. z테스트 선정하는데 가장 큰 어려움은 정규분포를 따르는지 모르는 거였음
        하지만 중심극한정리에 의해 크기가 충분히 크면 정규분포를 따른다

In [245]:
# 검정통계량 계산

(5.951351351351351 - 6.277816455695082) / (0.20077736867761145 / 37**0.5)

-9.890605286996074

정규분포표에는 -4까지 나옴
숫자가 크다는건 그만큼 자명한 사실이라는 의미?????????????


이유 분석


In [74]:
monthly_basis_score = df.groupby(df['Premiere'].dt.month)['IMDB Score'].mean()
monthly_basis_score

Premiere
1     6.002703
2     6.117949
3     6.322917
4     6.049206
5     6.213208
6     6.582857
7     6.091176
8     5.951351
9     6.481132
10    6.522078
11    6.398246
12    6.274510
Name: IMDB Score, dtype: float64

가정

1. 장르가 영향을 미쳤다
    - 8월의 장르 비율은 나머지의 비율과 달라야 한다.
2. 런타임이 영향을 미쳤다
3. 언어가 영향을 미쳤다
4. 그 외

In [8]:
# 1-1 8월의 장르

august_genre = df[df['Premiere'].dt.month == 8]['Genre'].value_counts()

august_genre_big = august_genre[august_genre > 1]
august_genre_small = august_genre[august_genre <= 1].sum()

august_genre_big

Genre
Documentary        9
Comedy             8
Drama              5
Thriller           2
Romantic comedy    2
Name: count, dtype: int64

In [ ]:
plt.pie(august_genre)

In [14]:
# 1-2 나머지의 장르

rest_genre = df[df['Premiere'].dt.month != 8]['Genre'].value_counts()

rest_genre

Genre
Documentary                             150
Drama                                    72
Comedy                                   41
Romantic comedy                          37
Thriller                                 31
                                       ... 
Comedy/Horror                             1
Romantic comedy-drama                     1
Political thriller                        1
Superhero-Comedy                          1
Animation/Christmas/Comedy/Adventure      1
Name: count, Length: 111, dtype: int64

In [ ]:
plt.pie(rest_genre)

1. 장르 비슷함 -> 무의미

============================

2. 런타임이 영향을 미쳤다

In [16]:
# 8월 런타임과 나머지 런타임
august_runtime = df[df['Premiere'].dt.month == 8]['Runtime'].mean()
rest_runtime = df[df['Premiere'].dt.month != 8]['Runtime'].mean()

In [22]:
df.groupby(df['Premiere'].dt.month)['Runtime'].mean()

Premiere
1      94.459459
2      96.410256
3      94.229167
4      94.380952
5      92.150943
6     101.171429
7      96.882353
8      86.972973
9      87.264151
10     94.324675
11     95.017544
12     91.843137
Name: Runtime, dtype: float64

2. 런타임이 짧다!

=============

3. 언어가 영향을 미쳤다

In [31]:
august_language = df[df['Premiere'].dt.month == 8]['Language'].value_counts()
august_language

Language
English             26
Hindi                4
Spanish              3
French               2
English/Japanese     1
Indonesian           1
Name: count, dtype: int64

In [28]:
rest_language = df[df['Premiere'].dt.month != 8]['Language'].value_counts()

In [ ]:
plt.pie(august_language)

In [ ]:
plt.pie(rest_language)

3. 언어 차이 없음

erwer